<h1><center> 
    CPT_S 434/534 Neural network design and application
</center></h1>

<!-- <h2><center>CPT_S 434/534</center></h2> -->

<h2><center>HW 1: ML -- code (80 pts)</center></h2>

### Name: *[SAIMAN DAHAL]*

### Student ID: *[11873444]*

## This assignment includes:

## Coding in Python (pytorch): train a regularized Softmax classifier on MNIST 

### Step 0: Install and configure: python ([Anaconda platform](https://docs.anaconda.com/anaconda/install/) recommended), [Jupyter Notebook](https://jupyter.org/install), [pytorch](https://pytorch.org/get-started/), [torchvision](https://anaconda.org/pytorch/torchvision), etc.

**Remark 1.** [Colab](https://colab.research.google.com) is a cloud platform that enables your Jupyter Notebooks (including this .ipynb assignment) to run with different runtime types (hardware acceleration is possible using GPU or TPU). You may also choose Colab to finish assignments (future assignments may require extensive computation that may be time-consuming on your laptop such as 1 or 2 hours). 

**Remark 2.** If you use Colab, make sure the working path/directory is the current one. See [this page](https://stackoverflow.com/questions/48298146/changing-directory-in-google-colab-breaking-out-of-the-python-interpreter).

**Remark 3.** If you use Colab, it is still required to convert your .ipynb to .html and submit **BOTH** files to Canvas. See [this page](https://stackoverflow.com/questions/53460051/convert-ipynb-notebook-to-html-in-google-colab) on how to convert to .html using Colab

### Step 1: Read the [provided document](CPTS434_534_HW1_note.pdf) to understand how to train a regularized Softmax classifier:

**Remark 1.** What is the problem setting of the multiclass classification

**Remark 2.** What is Cross-Entropy loss and $\color{red}{\text{the objective function of the regularized Softmax classifier}}$

**Remark 3.** How to compute the gradients of the objective function w.r.t. the weight matrix/vectors

**Remark 4.** What is the general algorithmic process of mini-batch SGD

### Step 2: Read provided code (with pytorch) to understand the logic of design and target, so that you know how to implement in the following step and incorporate your code into the provided code

### Step 3: Complete the code

#### 1) "mini_batch_gradient()" (**36 pts**) for computing mini-batch stochastic gradients in MB-SGD, and train the linear model with setting 1

#### 2) write the configure files for other 6 new settings and train their classifiers (4 pts each, **24 pts** in total). 

### Step 4: Record and plot results to show the loss and accuracy convergence (against #epoch)

### Step 5: Answer the five questions at the end of this file (4 pts each, **20 pts** in total)

## $\color{red}{\text{Submission:}}$

* Convert the .ipynb file to .html file (**save the execution outputs** to show your progress: otherwise grading may be affected)
    
* Upload **both** your .ipynb and .html files to Canvas separately (not zip them to a single zip file).

* Plots should be clear and easy to read.

## 1. (Read and run) Load/Download MNIST dataset using pytorch

In [1]:
import matplotlib.pyplot as plt
import numpy
from torchvision import datasets, transforms

train_set = datasets.MNIST('./data', train=True, download=True)
test_set = datasets.MNIST('./data', train=False, download=True)

x_train = train_set.data.numpy()
x_train = x_train.reshape(len(x_train),-1)
x_test = test_set.data.numpy()
x_test = x_test.reshape(len(x_test),-1)

y_train = train_set.targets.numpy()
y_test = test_set.targets.numpy()

print('Shape of x_train: ' + str(x_train.shape))
print('Shape of x_test: ' + str(x_test.shape))

print('Shape of y_train: ' + str(y_train.shape))
print('Shape of y_test: ' + str(y_test.shape))

Shape of x_train: (60000, 784)
Shape of x_test: (10000, 784)
Shape of y_train: (60000,)
Shape of y_test: (10000,)


## 2. (Read and run) Scale features

In [2]:
# calculate mu and sig using the training set
d = x_train.shape[1]
mu = numpy.mean(x_train, axis=0).reshape(1, d)
sig = numpy.std(x_train, axis=0).reshape(1, d)

# transform the training features
x_train = (x_train - mu) / (sig + 1E-6)

# transform the test features
x_test = (x_test - mu) / (sig + 1E-6)


## 3. (Read and run) Visualize a random image

In [ ]:
d = x_train.shape[0]
random_index = numpy.random.randint(d, size=1)[0]
plt.imshow(x_train[random_index].reshape(28,28))
print('Its label is')
print(y_train[random_index])

Its label is
3


## 4. (Read and run) Define softmax function and cross entropy loss

In [3]:
def softmax(z):
    """implement the softmax functions
    input: numpy ndarray
    output: numpy ndarray
    """
    exp_list = numpy.exp(z)
    result = 1/sum(exp_list) * exp_list
    result = result.reshape((len(z),1))
    assert (result.shape == (len(z),1))
    return result

def neg_log_loss(pred, label):
    """implement the negative log loss"""
    loss = -numpy.log(pred[int(label)])
    return loss




## 5. (Read and run) Functions for implementing linear softmax classifier

In [4]:
import os
import sys
import json
import numpy

root = os.path.abspath('.')
root += '/configs/'

def loadConfig(name):
    """ Read a configuration file as a dictionary"""
    full_path = root + name
    json_file = open(full_path, 'r')
    cfg = json.load(json_file)
    json_file.close()
    return cfg  

def initialize(num_inputs,num_classes):
    """initialize the parameters"""
    # num_inputs = 28*28 = 784
    # num_classes = 10
    w = numpy.zeros((num_classes, num_inputs)) # (10*784)
    
    param = {
        'w' : w, # (10*784)
    }
    return param

def eval(param, hyp, x_data, y_data):
    """ implement the evaluation function
    input: param -- parameters dictionary (w)
           hyp -- hyper-parameter: we use hyp['lambda'] to compute regularization
           x_data -- x_train or x_test (size, 784)
           y_data -- y_train or y_test (size,)
    output: loss and accuracy
    """
    
    reg_lambda = hyp['lambda']
    
    # w: (10*784), x: (10000*784), y:(10000,)
    loss_list = []
    w = param['w'].transpose()
    dist = numpy.array([numpy.squeeze(softmax(numpy.matmul(x_data[i], w))) for i in range(len(y_data))])

    result = numpy.argmax(dist,axis=1)
    accuracy = sum(result == y_data)/float(len(y_data))

    loss_list = [neg_log_loss(dist[i],y_data[i]) for i in range(len(y_data))]
    loss = sum(loss_list) / len(loss_list) + reg_lambda/2 * numpy.sum(w * w)
    return loss, accuracy

def train(param, hyp, x_train, y_train, x_test, y_test,cfg_idx):
    """ implement the train function
    input: param -- parameters dictionary (w)
           hyp -- hyperparameters dictionary
           x_train -- (60000, 784)
           y_train -- (60000,)
           x_test -- x_test (10000, 784)
           y_test -- y_test (10000,)
    output: train_loss_list, train_acc_list, test_loss_list, test_acc_list
           Four lists contain the epoch-wise loss function on training data, accuracy on training data, loss function on testing data, accuracy on testing data, respectively
    """
    num_epoches = hyp['num_epoches']
    batch_size = hyp['batch_size']
    learning_rate = hyp['learning_rate']
    mu = hyp['mu']
    reg_lambda = hyp['lambda']
    train_loss_list, train_acc_list, test_loss_list, test_acc_list = [],[],[],[]
    if bool(hyp['momentum']) == True:
        w_velocity = numpy.zeros(param['w'].shape)
        b_velocity = numpy.zeros(param['b'].shape) 

    for epoch in range(num_epoches):
        
        # select the random sequence of training set
        rand_indices = numpy.random.choice(x_train.shape[0],x_train.shape[0],replace=False)
        num_batch = int(x_train.shape[0]/batch_size)
        
        if bool(hyp['learning_decay']) == True:
            try:
                if test_acc_list[-1] - test_acc_list[-2] < 0.001:
                    learning_rate *= hyp['decay_factor']
            except:
                pass
            
            message = 'learning rate: %.8f' % learning_rate
            print(message)
            logging.info(message)

        # for each batch of train data
        for batch in range(num_batch):
            index = rand_indices[batch_size*batch:batch_size*(batch+1)]
            x_batch = x_train[index]
            y_batch = y_train[index]

            # calculate the stochastic gradient w.r.t w 
            dw, batch_loss = mini_batch_gradient(param, x_batch, y_batch, reg_lambda)

            param['w'] -= learning_rate * dw
            
            if (batch+1) % 100 == 0:
                message = 'Epoch [%d/%d], Batch [%d/%d], Loss %.4f' % (epoch+1, num_epoches, batch+1, num_batch, batch_loss)
                print(message)

        train_loss, train_acc = eval(param,hyp,x_train,y_train)
        test_loss, test_acc = eval(param,hyp,x_test,y_test)
        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        test_loss_list.append(test_loss)
        test_acc_list.append(test_acc)

        message = 'Epoch %d/%d, Train Loss %.4f, Train Acc %.4f, Test Loss %.4f, Test Acc %.4f' % (epoch+1, num_epoches, train_loss, train_acc, test_loss, test_acc)
        print(message)
        logging.info(message)
    return train_loss_list, train_acc_list, test_loss_list, test_acc_list


def plot(train_loss_list, train_acc_list, test_loss_list, test_acc_list, cfg_idx):
    """store the plots"""
    # epoch_list = list(range(len(loss_list)))
    plt.plot(train_loss_list, '-b', label='train loss')
    plt.plot(test_loss_list, '-r', label='test loss')
    plt.legend()
    plt.ylabel('Loss Function')
    plt.xlabel('Epoch')
    plt.xticks(rotation=60)
    plt.title('Loss Function ~ Epoch')
    plt.savefig('assets/loss_{}.png'.format(cfg_idx))
    plt.show()
    
    plt.plot(train_acc_list, '-b', label='train acc')
    plt.plot(test_acc_list, '-r', label='test acc')
    plt.legend()
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.xticks(rotation=60)
    plt.title('Accuracy ~ Epoch')
    plt.savefig('assets/accr_{}.png'.format(cfg_idx))
    plt.show()
    

def main(cfg_idx): 
#     cfg_idx = args.config
    cfg_name = 'config_{}.json'.format(cfg_idx)
    hyperpara = loadConfig(cfg_name)

    # setting the random seed
    numpy.random.seed(1024)

    # initialize the parameters
    num_inputs = x_train.shape[1]
    num_classes = len(set(y_train))
    param = initialize(num_inputs,num_classes)

    # train the model
    train_loss_list, train_acc_list, test_loss_list, test_acc_list = train(param,hyperpara,x_train,y_train,x_test,y_test, cfg_idx)

    # plot the loss and accuracy
    plot(train_loss_list, train_acc_list, test_loss_list, test_acc_list, cfg_idx)

## 6. (To finish, 36 pts) Implementation of mini-batch gradients

In [11]:
def mini_batch_gradient(param, x_batch, y_batch, reg_lambda):
    """Compute the mini-batch gradient.

    Args:
        param (dict): Parameters dictionary containing 'w'.
        x_batch (numpy.ndarray): Input data for the mini-batch.
        y_batch (numpy.ndarray): Labels for the mini-batch.
        reg_lambda (float): Regularization parameter.

    Returns:
        numpy.ndarray: Gradient w.r.t. weight 'w'.
        float: Average loss on the mini-batch samples.
    """
    weight = param['w']
    distance=numpy.array([numpy.squeeze(softmax(numpy.matmul(x_batch[i],weight.T))) for i in range(len(y_batch))])
    list_=[neg_log_loss(distance[i],y_batch[i]) for i in range (len(y_batch))]
    
    loss=sum(list_)
    avg=loss/len(x_batch)

    # 
    regularization=(reg_lambda/2)*sum(sum(numpy.matmul(weight,weight.T)))
    batch_loss=loss+regularization

    # 
    d_batch=[]
    for data_index, label_weights in enumerate(dist):
        d_i=[]
        for class_index, weight_ in enumerate(label_weights):
            if (y_batch[data_index]==class_index):
                d_i.append(numpy.matmul((weight_-1), x_batch[data_index]))
            else:
                d_i.append(numpy.matmul((weight_-0), x_batch[data_index]))

        d_batch.append(numpy.array(d_i))


    dw_=[]
    for weight_ in weight:
        dw_.append(weight=reg_lambda)

    dw=sum(numpy.array(d_batch))/length(x_batch) + numpy.array(dw_)

    

    return dw, batch_loss


## 7. (Read and run) Train your model using the provided configuration "setting 1" (in *configs/config_setting_1.json*) untill convergence 

In [1]:
if os.path.isdir('logs') == False:
    os.mkdir('logs')
    
if os.path.isdir('assets') == False:
    os.mkdir('assets')


if __name__ == "__main__":

    cfg_idx = 'setting_1'

    
    import logging
    logging.basicConfig(filename="./logs/{}.log".format(cfg_idx), filemode="w", format="%(message)s", level=logging.DEBUG)
    
    main(cfg_idx)

NameError: name 'os' is not defined

In [5]:
import numpy as np

## 8. (To finish, 4 pts) Create a new configuration file for setting 2 (see comment below) and train your model

In [ ]:
if __name__ == "__main__":

    cfg_idx = 'setting_2'

# configuration in setting 2:
#     {
#     "num_epoches" : 50,
#     "batch_size" : 600,
#     "learning_rate" : 0.1,
#     "learning_decay" : 0,
#     "decay_factor" : 0.75,
#     "momentum" : 0,
#     "lambda" : 0.00,
#     "mu" : 0.9
# }
    
    import logging
    logging.basicConfig(filename="./logs/{}.log".format(cfg_idx), filemode="w", format="%(message)s", level=logging.DEBUG)
    
    main(cfg_idx)    

## 9. (To finish, 4 pts) Create a new configuration file for setting 3 (see comment below) and train your model

In [ ]:
if __name__ == "__main__":

    cfg_idx = 'setting_3'
    
# configuration in setting 3:
#     {
#     "num_epoches" : 25,
#     "batch_size" : 100,
#     "learning_rate" : 0.1,
#     "learning_decay" : 0,
#     "decay_factor" : 0.75,
#     "momentum" : 0,
#     "lambda" : 0.00,
#     "mu" : 0.9
# }
    
    
    import logging
    logging.basicConfig(filename="./logs/{}.log".format(cfg_idx), filemode="w", format="%(message)s", level=logging.DEBUG)
    
    main(cfg_idx)
    

## 10. (To finish, 4 pts) Create a new configuration file for setting 4 (see comment below) and train your model

In [ ]:
if __name__ == "__main__":

    cfg_idx = 'setting_4'
    
# configuration in setting 4:
# {
#     "num_epoches" : 25,
#     "batch_size" : 100,
#     "learning_rate" : 0.01,
#     "learning_decay" : 0,
#     "decay_factor" : 0.75,
#     "momentum" : 0,
#     "lambda" : 0.00,
#     "mu" : 0.9
# }
    
    
    import logging
    logging.basicConfig(filename="./logs/{}.log".format(cfg_idx), filemode="w", format="%(message)s", level=logging.DEBUG)
    
    main(cfg_idx)

## 11. (To finish, 4 pts) Create a new configuration file for setting 5 (see comment below) and train your model

In [ ]:
if __name__ == "__main__":

    cfg_idx = 'setting_5'
    
# configuration in setting 5:
#     {
#     "num_epoches" : 50,
#     "batch_size" : 600,
#     "learning_rate" : 0.1,
#     "learning_decay" : 0,
#     "decay_factor" : 0.75,
#     "momentum" : 0,
#     "lambda" : 0.001,
#     "mu" : 0.9
# }
    
    
    import logging
    logging.basicConfig(filename="./logs/{}.log".format(cfg_idx), filemode="w", format="%(message)s", level=logging.DEBUG)
    
    main(cfg_idx)

## 12. (To finish, 4 pts) Create a new configuration file for setting 6 (see comment below) and train your model

In [ ]:
if __name__ == "__main__":

    cfg_idx = 'setting_6'
    
# configuration in setting 6:
#     {
#     "num_epoches" : 50,
#     "batch_size" : 600,
#     "learning_rate" : 0.1,
#     "learning_decay" : 0,
#     "decay_factor" : 0.75,
#     "momentum" : 0,
#     "lambda" : 0.01,
#     "mu" : 0.9
# }
    
    import logging
    logging.basicConfig(filename="./logs/{}.log".format(cfg_idx), filemode="w", format="%(message)s", level=logging.DEBUG)
    
    main(cfg_idx)

## 13. (To finish, 3 pts) Create a new configuration file for setting 7 (see below) and train your model

In [ ]:
if __name__ == "__main__":

    cfg_idx = 'setting_7'
    
# configuration in setting 7:
#     {
#     "num_epoches" : 50,
#     "batch_size" : 600,
#     "learning_rate" : 0.1,
#     "learning_decay" : 0,
#     "decay_factor" : 0.75,
#     "momentum" : 0,
#     "lambda" : 0.1,
#     "mu" : 0.9
# }
    
    
    import logging
    logging.basicConfig(filename="./logs/{}.log".format(cfg_idx), filemode="w", format="%(message)s", level=logging.DEBUG)
    
    main(cfg_idx)

## 14. (To finish, 4 pts each question, 20 pts in total) Answer the following five questions

### Q1 (4 pts): What can we learn by comparing results of setting 1 and setting 2? Please analyse your observation/conclusion

**Answer**:




### Q2 (4 pts): What can we learn by comparing results of setting 2 and setting 3? Please analyse your observation/conclusion

**Answer**:




### Q3 (4 pts): What can we learn by comparing results of setting 3 and setting 4? Please analyse your observation/conclusion

**Answer**:



### Q4 (4 pts): What can we learn by comparing results of setting 2 and 5 (or 6)? Please analyse your observation/conclusion

**Answer**:



### Q5 (4 pts): What may happen to setting 7, compared to settings 2, 5 and 6?

**Answer**:



In [2]:
# convert this file "HW1_to_Canvas.ipynb" to "HW1_to_Canvas.html"
!jupyter nbconvert --to html HW1_to_Canvas.ipynb 

[NbConvertApp] Converting notebook HW1_to_Canvas.ipynb to html
[NbConvertApp] Writing 656244 bytes to HW1_to_Canvas.html
